# Chatbot Projesi
Bu proje, Python programlama dili kullanarak bir chatbot oluşturmayı amaçlamaktadır. Chatbot, kullanıcılarla insan benzeri etkileşimler kurarak, işletmelerin günlük olarak aldıkları en yaygın soruları çözmelerine yardımcı olur. Bu çalışma, başlangıç seviyesindeki geliştiriciler için tasarlanmıştır ve NLTK kütüphanesi kullanılarak bir chatbot'un nasıl oluşturulacağını adım adım öğreneceksiniz.

In [ ]:
Chatbot Projesi
Bu proje, Python programlama dili kullanarak bir chatbot oluşturmayı amaçlamaktadır. Chatbot, kullanıcılarla insan benzeri etkileşimler kurarak, işletmelerin günlük olarak aldıkları en yaygın soruları çözmelerine yardımcı olur. Bu çalışma, başlangıç seviyesindeki geliştiriciler için tasarlanmıştır ve NLTK kütüphanesi kullanılarak bir chatbot'un nasıl oluşturulacağını adım adım öğreneceksiniz.
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import json
import random
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Merhaba", "Selam", "Nasılsın?", "İyi günler", "Günaydın", "İyi akşamlar"],
      "responses": ["Merhaba! Nasıl yardımcı olabilirim?", "Selam! Size nasıl destek olabilirim?", "İyi günler! Sorunuz nedir?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Görüşürüz", "Hoşça kal", "Bay bay", "Sonra görüşürüz", "Çıkış yap"],
      "responses": ["Görüşmek üzere!", "Hoşça kalın! İyi günler.", "Yardıma ihtiyacınız olursa buradayım!"]
    },
    {
      "tag": "teşekkür",
      "patterns": ["Teşekkürler", "Sağ ol", "Çok yardımcı oldun", "Harikasın", "Mükemmelsin"],
      "responses": ["Rica ederim!", "Ne demek, her zaman!", "Yardımcı olabildiysem ne mutlu bana!"]
    },
    {
      "tag": "hava_durumu",
      "patterns": ["Hava nasıl?", "Yağmur yağacak mı?", "Bugün sıcaklık kaç derece?", "Rüzgar hızı nedir?"],
      "responses": ["Şu anda hava güneşli ve 25°C.", "Yağmur beklenmiyor, açık bir gün olacak.", "Sıcaklık 22°C ile 28°C arasında değişiyor."]
    },
    {
      "tag": "randevu_alma",
      "patterns": ["Randevu almak istiyorum", "Doktor randevusu", "Müsait zamanınız var mı?", "Bugün randevu alabilir miyim?"],
      "responses": ["Randevu için lütfen web sitemizi ziyaret edin.", "En yakın müsait tarih yarın saat 14:00."]
    },
    {
      "tag": "ürün_bilgisi",
      "patterns": ["Bu ürün ne kadar?", "İndirim var mı?", "Stok durumu nedir?", "Ürün özelliklerini görebilir miyim?"],
      "responses": ["Bu ürünün fiyatı 499 TL.", "Şu anda %20 indirim uygulanıyor!", "Stoklarımızda mevcut."]
    },
    {
      "tag": "şikayet",
      "patterns": ["Şikayetim var", "Ürün kusurlu geldi", "Hizmetinizden memnun değilim", "İade süreci nasıl işler?"],
      "responses": ["Üzgünüz, sorununuzu çözmek için buradayız.", "Lütfen destek@firma.com adresine e-posta gönderin."]
    },
    {
      "tag": "mizah",
      "patterns": ["Espri yap", "Komik bir şey söyle", "Güldür beni", "Mizah yeteneğin var mı?"],
      "responses": ["Neden tavuklar karşıdan karşıya geçti? Yolun diğer tarafına gitmek için!", "Bir yapay zeka olarak komik olmaya çalışıyorum. Başarılı oldum mu?"]
    },
    {
      "tag": "saat",
      "patterns": ["Saat kaç?", "Zamanı söyle", "Günün saati nedir?"],
      "responses": ["Şu an saat 15:30.", "Saat 3'ü 30 geçiyor."]
    },
    {
      "tag": "yardım",
      "patterns": ["Yardım lazım", "Nasıl kullanılır?", "Destek almak istiyorum", "Bana rehberlik eder misin?"],
      "responses": ["Size nasıl yardımcı olabilirim?", "Lütfen sorunuzu detaylandırın."]
    }
  ]
}
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
# JSON dosyasını yükle
with open('intents.json.txt.txt', encoding='utf-8') as file:
    intents = json.load(file)

words = []
classes = []
documents = []

# Cümleleri tokenize et ve köklerine ayır
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ["?", "!"]]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))  # Düzeltildi
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    results = [[i, r] for i, r in enumerate(res) if r > 0.25]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Sohbeti başlat
print("Chatbot aktif! (Çıkmak için 'quit' yazın)")
while True:
    user_input = input("Sen: ")
    if user_input.lower() == "quit":
        break
    intents_list = predict_class(user_input)
    response = get_response(intents_list, intents)
    print("Bot:", response)

Chatbot aktif! (Çıkmak için 'quit' yazın)
Sen:  merhaba
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Bot: Selam! Size nasıl destek olabilirim?
Sen:  nasılsın
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Bot: İyi günler! Sorunuz nedir?
Sen:  hava nasıl
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: Şu anda hava güneşli ve 25°C.
Sen:  teşşekkür edrim
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: Merhaba! Nasıl yardımcı olabilirim?
Sen:  görüşürüz
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Bot: Hoşça kalın! İyi günler.
Sen:  quit
import pickle

with open('chatbot_model.pkl', 'wb') as f:
    pickle.dump(model, f)



In [2]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import json
import random

In [3]:
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BETÜL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BETÜL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# JSON dosyasını yükle
with open('intents.json.txt.txt', encoding='utf-8') as file:
    intents = json.load(file)

words = []
classes = []
documents = []

# Cümleleri tokenize et ve köklerine ayır
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ["?", "!"]]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [5]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))  # Düzeltildi
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200


C:\Users\BETÜL\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0870 - loss: 2.3313      
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1384 - loss: 2.2698 
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1535 - loss: 2.2446 
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1522 - loss: 2.3241     
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1927 - loss: 2.1497 
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2775 - loss: 2.1904 
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4027 - loss: 2.1123 
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3217 - loss: 2.0849 
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3370 - loss: 2.0252 
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3849 - loss: 2.0897 
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4459 - loss: 1.9338 
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5383 - loss: 

In [7]:
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    results = [[i, r] for i, r in enumerate(res) if r > 0.25]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Sohbeti başlat
print("Chatbot aktif! (Çıkmak için 'quit' yazın)")
while True:
    user_input = input("Sen: ")
    if user_input.lower() == "quit":
        break
    intents_list = predict_class(user_input)
    response = get_response(intents_list, intents)
    print("Bot:", response)

Chatbot aktif! (Çıkmak için 'quit' yazın)


Sen:  merhaba


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Bot: Selam! Size nasıl destek olabilirim?


Sen:  nasılsın


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Bot: İyi günler! Sorunuz nedir?


...

Sen:  teşşekkür edrim


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: Merhaba! Nasıl yardımcı olabilirim?


Sen:  görüşürüz


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Bot: Hoşça kalın! İyi günler.


Sen:  quit


In [15]:
import pickle

with open('chatbot_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential


# Save the model in H5 format
model.save('chatbot_model.h5')
print("Model successfully converted and saved as chatbot_model.h5")

Model successfully converted and saved as chatbot_model.h5
